# EE 304 - Neuromorphics: Brains in Silicon


##  Implementing Synaptic Weights

The story thus far: 

We reviewed how much the four different architectures increase energy consumption and reduce peak throughput in order to save area by sharing hardware elements among neural elements. 

We found that the Shared Dendrite Architecture was the only architecture that could match the Fully Dedicated Architure's AET cost. It amortized its energy and time cost by delivering synpatic input to $O(\surd N)$ neurons neighboring the one targeted and using $O(\surd N)$ RAM banks to store the weights.

####  Outline for this lecture

We will study how to implement synaptic weights when architectures that use shared synapses (Shared Synapse and Shared Dendrite).

We will find that a probabilistic approach is compatible with shared synapses. 

However, this compatibility comes at the cost of trading a linear scaling of SNR with spike rate for a square-root scaling.

## Deterministic Approach

Consider a group of neurons (indexed by $j$) connected to another group of neurons (indexed by $i$) with a weight matrix ${\bf W}$ (indexed by $ij$). Given that neuron $j$ fires its $k^{\rm th}$ spike at $t^j_k$, the input current neuron $i$ receives at any time $t$ is expected to be  

\begin{eqnarray*}
    E[I_i(t)] 
           & = & {\rm E}\!\!\left[\sum_j w_{ij}\left(h(t)\!*\!\!\sum_k \delta(t-t_k^j)\right)\right]\\
           & = & \sum_j w_{ij}{\rm E}\!\!\left[\left(\sum_k h(t-t_k^j)\right)\right]\\
           & = & \sum_j w_{ij}\lambda_j
\end{eqnarray*}

where $\lambda_j$ is neuron $j$'s mean spike-rate over a interval of time whose length accounts for most of $h(t)$'s area. This result follows because $h(t)$ has unit area. These unit-areas sum together to produce a total area equal to the number of spikes observed over the said interval. Hence, the total area divided by the elapsed time (i.e. the average value) will be equal to the mean spike-rate. For decaying exponential synapses, an interval of $3\tau$ captures 95% of the $h(t)$'s area.  

### Variance of Input Current for Uniform Spike Trains  

<img src="files/lecture12/SumOfUniformSpikeTrains.png" width="1000">

Assuming neuron $j$'s spike-train is described by a uniform point-process, the variance in neuron $i$'s input current is
\begin{eqnarray*}
    {\rm Var}[I_i(t)] 
    & = & {\rm Var}\!\!\left[\sum_j w_{ij}\left(h(t)\!*\!\!\sum_k \delta(t-t_k^j)\right)\right]\\
    & = & \sum_j w_{ij}^2{\rm Var}\!\!\left[\left(\sum_k h(t-t_k^j)\right)\right]\\
    & = & \sum_j w_{ij}^2\!\!\left\{\lambda_j^2\left(\frac{1}{2\tau \lambda_j}\coth\!\!\left(\!\frac{1}{2\tau\lambda_j}\!\right)-1\right)\right\}
\end{eqnarray*}
when the spikes are filtered by decaying-exponential synapses with time-constant $\tau$. For details, see <a href="http://nbviewer.ipython.org/github/fragapanagos/notebooks/blob/master/synapse/synapse_SNR_uniform.ipynb">Sam's Uniform Spikes Notebook</a>. The expression in squiggly brackets is plotted above (with the $j$ index dropped). Note that it increases linearly for $\tau\lambda<0.1$ and asymtotes to $1/12\tau^2$ ($\approx0.0833/\tau^2$) when $\tau\lambda>1$.

### Asymptotic Approximations for High and Low Spike-Rates

In the two asymptotic cases, where neuron $j$ fires much more or much less than one spike in $\tau$ seconds, the variance's expression can be simplified:

$$
{\rm Var}[I_i(t)] \approx 
\left\{ 
  \begin{matrix}
    \frac{1}{12\tau^2}\!\sum_j w_{ij}^2 & & \tau\lambda_j \gg 1\\
    \frac{1}{2\tau}\!\sum_j \lambda_j w_{ij}^2 & & \tau \lambda_j \ll 1
  \end{matrix} \right.
$$

To gain some intuition for what is going on, assume all the weights are equal ($w_{ij}=w$) and compute the signal-to-noise ratio (SNR), defined as the mean divided by the standard deviation. In both cases, you will find that the SNR depend only on the total spike-rate, which you can express as the product of the population's average spike-rate, $\lambda$, and its neuron count, $N$: 

- For the high spike-rate regime 
$$
    \frac{{\rm E}[I_i(t)]}{\sqrt{{\rm Var}[I_i(t)]}} 
    \approx \frac{w\!\sum_j\!\lambda_j}{\sqrt{N w^2/(12\tau^2)}} 
    = 2\tau\sqrt\frac{3}{N}\!\sum_{j=1}^{N}\!\lambda_j
    = 2\tau\lambda\sqrt{3N}
$$
    - Increases linearly with $\lambda$, the average spike-rate
    - Increases as the square-root of $N$, the neuron count 

- For the low spike-rate regime 
$$
    \frac{{\rm E}[I_i(t)]}{\sqrt{{\rm Var}[I_i(t)]}} 
    \approx \frac{w\!\sum_j\!\lambda_j}{\sqrt{w^2\!\sum_j\!\lambda_j/(2\tau)}} 
    = \sqrt{2\tau}\surd\!\sum_{j=1}^{N}\!\lambda_j
    = \sqrt{2\tau\lambda N}
$$
    - Increases as the square-root of $\lambda$, the average spike-rate
    - Increases as the square-root of $N$, the neuron count

Note that, in the low-rate case, the SNR increases as the square-root of the **total** spike-rate. Thus, doubling the neuron count while keeping the average spike-rate the same is equivalent to doubling the average spike-rate while keeping the neuron count the same. 

## Probabilistic Approach

An alternative approach to implementing synaptic weights is to deliver neuron $j$'s spike to neuron $i$ with probability equal to $w_{ij}$. Spikes that get through are weighted equally, making this scheme compatible with hardware that uses shared-synapse circuits. Let  $I_{ij}^k \in \{0,1\}$ indicate that neuron $j$'s $k^{\rm th}$ spike made it through to neuron $i$. In which case, the current neuron $i$ receives at any time $t$ is expected to be  

\begin{eqnarray*}
    E[I_i(t)] 
           & = & {\rm E}\!\!\left[\sum_j \left(h(t)\!*\!\!\sum_k \!I_{ij}^k\delta(t-t_k^j)\right)\right]\\
           & = & \sum_j {\rm E}\!\!\left[\left(\sum_k \!I_{ij}^kh(t-t_k^j)\right)\right]\\
           & = & \sum_j {\rm E}[I_{ij}^k]\lambda_j
\end{eqnarray*}

where $\lambda_j$ is neuron $j$'s mean spike-rate (defined above). Thus, neuron $j$'s spike-rate is weighted correctly if $I_{ij}^k$ takes on the value 1 with probability $p = w_{ij}$ and value 0 with probability $p = 1-w_{ij}$. Mathematicians have a name for this process: p-thinning. Note that, for this to work, all the weights must be smaller than 1. If this is not the case, they have to be rescaled  appropriately and the scale factor accounted for in the sum above.   

### Variance of Input Current for $p$-thinned Uniform Spike Trains  

<img src="files/lecture12/p-thinned-uniform_09.png" width="1000">
<img src="files/lecture12/p-thinned-uniform_05.png" width="1000">
<img src="files/lecture12/p-thinned-uniform_01.png" width="1000">

Assuming neuron $j$'s spike-train, with uniform rate $\lambda_j$, is $w_{ij}$-thinned, the variance in neuron $i$'s input current is
\begin{eqnarray*}
    {\rm Var}[I_i(t)] 
    & = & {\rm Var}\!\!\left[\sum_j \left(h(t)\!*\!\!\sum_k I_{ij}^k\delta(t-t_k^j)\right)\right]\\
    & = & \sum_j {\rm Var}\!\!\left[\left(\sum_k I_{ij}^k h(t-t_k^j)\right)\right]\\
    & = & \sum_j w_{ij}^2\lambda_j^2\!
        \left\{
            \frac{1}{2\tau w_{ij}\lambda_j}
            \left(w_{ij}\coth\!\!\left(\!\frac{1}{2\tau\lambda_j}\!\right)-w_{ij}+1\right) - 1
        \right\}
\end{eqnarray*}
when the spikes are filtered by decaying-exponential synapses with time-constant $\tau$. For details, see <a href="http://nbviewer.ipython.org/github/fragapanagos/notebooks/blob/master/synapse/synapse_SNR_uniform_pthinned.ipynb">Sam's p-Thinned Uniform Spikes Notebook</a>. One term in the sum is plotted above (with the $w_{ij}$ replaced by $p=0.9, 0.5, \&\; 0.1$). For $p \leq 0.1$, the variance is more or less linearly proportional to the $p$-thinned uniform spike-train's rate (i.e. $w_{ij}\lambda_j$).

### Poisson Approximation